# Multiple flows

1. Flow 1 runs, inputs bar1 and outputs bar2
2. Flow 2 runs, inputs Flow1.input, Flow1.output

In [1]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

import pandas as pd
from doltpy.core import Dolt
from doltpy.core.write import import_df

dolt = Dolt.init(".")

df_v1 = pd.DataFrame({"A": [1,1,1], "B": [1,1,1]})
df_v2 = pd.DataFrame({"A": [1,1,1,2,2,2], "B": [1,1,1,2,2,2]})

import_df(dolt, "bar", df_v1.reset_index(), ["index"], "create")
dolt.add(".")
dolt.add("bar")
dolt.commit("Initialize bar")

v1 = list(dolt.log(number="1").keys())[0]

import_df(dolt, "bar", df_v2.reset_index(), ["index"], "update")
dolt.add("bar")
dolt.commit("Add rows to bar")

v2 = list(dolt.log(number="1").keys())[0]

01-18 11:24:19 doltpy.core.dolt INFO     Initializing Dolt repo in existing dir .
01-18 11:24:19 doltpy.core.dolt INFO     Creating a new repo in .
01-18 11:24:19 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
01-18 11:24:19 doltpy.core.write.write INFO     Importing to table bar in dolt directory located in ., import mode create
01-18 11:24:19 doltpy.core.dolt INFO     Rows Processed: 3, Additions: 3, Modifications: 0, Had No Effect: 0
01-18 11:24:19 doltpy.core.dolt INFO     
01-18 11:24:19 doltpy.core.dolt INFO     
01-18 11:24:19 doltpy.core.dolt INFO     commit 27p8h38rkdpcq7t0hcsc0c8kimurkd19
Author: Max Hoffman <maximilian.wolfgang1@gmail.com>
Date:   Mon Jan 18 11:24:19 -0800 2021

	Initialize bar


01-18 11:24:19 doltpy.core.write.write INFO     Importing to table bar in dolt directory located in ., import mode update
01-18 11:24:19 doltpy.core.dolt INFO     Rows Processed: 6, Additions: 3, Modifications: 0, Had No Effect: 3
01

In [2]:
!poetry run python3 demo_one.py run

Metaflow 2.2.5.post32+git1ea1770 executing MultiFlowDemo1 for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-18 11:24:50.103 Workflow starting (run-id 1610997890092238):
2021-01-18 11:24:50.113 [1610997890092238/start/1 (pid 35732)] Task is starting.
2021-01-18 11:24:51.968 [1610997890092238/start/1 (pid 35732)] 01-18 11:24:51 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-18 11:24:52.015 [1610997890092238/start/1 (pid 35732)] 01-18 11:24:52 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-18 11:24:52.237 [1610997890092238/start/1 (pid 35732)] 01-18 11:24:52 doltpy.core.dolt INFO     * master                                        	h3krq0mpuvdlv8li1jmvjnlv41ngq4hl
2021-01-18 11:24:52.416 [1610997890092238/start/1 (pid 35732)] 
2021-01-18 11:24:52.416 [1610997890092238/start/1 (pid 35732)] 01-18 11:24:52 dolt

2021-01-18 11:24:55.303 [1610997890092238/middle/2 (pid 35774)] Task finished successfully.
2021-01-18 11:24:55.310 [1610997890092238/end/3 (pid 35817)] Task is starting.
2021-01-18 11:24:56.430 [1610997890092238/end/3 (pid 35817)] 01-18 11:24:56 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
2021-01-18 11:24:56.437 [1610997890092238/end/3 (pid 35817)] 01-18 11:24:56 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-18 11:24:56.599 [1610997890092238/end/3 (pid 35817)] Task finished successfully.
2021-01-18 11:24:56.600 Done!
01-18 11:24:56 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-18 11:24:56 doltpy.core.system_helpers INFO     No processes to clean up, exiting


In [ ]:
# TODO: pluck flow from last step

In [4]:
!poetry run python3 demo_two.py run --flow-dep MultiFlowDemo1/1610997890092238

Metaflow 2.2.5.post32+git1ea1770 executing MultiFlowDemo2 for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-18 11:26:05.971 Workflow starting (run-id 1610997965963222):
2021-01-18 11:26:05.977 [1610997965963222/start/1 (pid 35837)] Task is starting.
2021-01-18 11:26:07.025 [1610997965963222/start/1 (pid 35837)] 01-18 11:26:07 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-18 11:26:07.117 [1610997965963222/start/1 (pid 35837)] 01-18 11:26:07 doltpy.core.dolt INFO     flow_name,run_id,step_name,task_id,kind,database,table_name,commit,timestamp
2021-01-18 11:26:07.123 [1610997965963222/start/1 (pid 35837)] MultiFlowDemo1,1610997890092238,start,1,read,.,bar,h3krq0mpuvdlv8li1jmvjnlv41ngq4hl,1.6109979e+09
2021-01-18 11:26:07.123 [1610997965963222/start/1 (pid 35837)] 
2021-01-18 11:26:07.123 [1610997965963222/start/1 (pid 35837)] 01-18 11:26:07 doltpy.core.dolt INFO  

2021-01-18 11:26:08.316 [1610997965963222/start/1 (pid 35837)] Task finished successfully.
2021-01-18 11:26:08.323 [1610997965963222/middle/2 (pid 35889)] Task is starting.
2021-01-18 11:26:09.672 [1610997965963222/middle/2 (pid 35889)] 01-18 11:26:09 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-18 11:26:09.705 [1610997965963222/middle/2 (pid 35889)] 01-18 11:26:09 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-18 11:26:09.767 [1610997965963222/middle/2 (pid 35889)] 01-18 11:26:09 doltpy.core.dolt INFO     * master                                        	t23vjks4005suj719a5psn9d9v8lthjg
2021-01-18 11:26:09.873 [1610997965963222/middle/2 (pid 35889)] 
2021-01-18 11:26:09.873 [1610997965963222/middle/2 (pid 35889)] <flow MultiFlowDemo2 step middle> failed:
2021-01-18 11:26:09.884 [1610997965963222/middle/2 (pid 35889)]     Internal error
2021-01-18 11:26:09.885 [161099796

In [6]:
!cat demo_two.py

import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

import pickle
import time

from metaflow import FlowSpec, step, DoltDT, Parameter
from metaflow.datatools.dolt import DoltRun
import pandas as pd
from sklearn import tree

class MultiFlowDemo2(FlowSpec):

    flow_dep = Parameter('flow-dep',  help="Specifc the tag for the input version", required=True)

    @step
    def start(self):
        flow, run = self.flow_dep.split("/")
        d = DoltRun(flow_name=flow, run_id=run)
        f_input = d.reads[0]
        f_output = d.writes[0]
        with DoltDT(run=self) as dolt:
            self.inp1 = dolt.read_table(f_input.table_name, commit=f_input.commit)
            self.inp2 = dolt.read_table(f_output.table_name, commit=f_output.commit)

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self) as dolt:

            df = self.inp1 + self.inp2



        self.next(self.end)

    @step
    def end(self):
        pass


if __